In [57]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\064810\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [58]:
pip install adjustText

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\064810\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [11]:
# Geospatial Visualization - Turkey Coordinates
# A lightweight notebook to plot grouped coordinate sets a/b/c/d on a Turkey basemap for quik visual inspetion and reporting.

""""
Created by: Batuhan Çakır
Contact: https://www.linkedin.com/in/batuhan-cakir/ 
Github: https://github.com/batucakir/
Last Updated: 20.01.2026
"""

import os
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from shapely.geometry import Point, box
from adjustText import adjust_text

# Copy Dir
os.chdir(os.path.expanduser("~/Downloads"))

def load_world_10m():
    url = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_10m_admin_0_countries.geojson"
    return gpd.read_file(url)

def norm_point(p):
    if isinstance(p, dict):
        lat=float(p.get("lat")); lon=float(p.get("lon") or p.get("lng"))
        name=str(p.get("name","")); grp=(p.get("group") or "").lower() or None
    else:
        lat,lon,name = float(p[0]), float(p[1]), str(p[2])
        grp = (str(p[3]).lower() if len(p)>=4 and p[3] is not None else None)
        
    # lat-lon checking if not true 
    if 25 <= lat <= 46.5 and 34 <= lon <= 43.8:
        lon, lat = lat, lon
    if grp not in {"a_class","b_class","c_class","d_class"}:
        grp = "a_class"
    return lat, lon, name, grp

def make_gdf(points):
    rows=[norm_point(p) for p in points]
    geoms=[Point(lon,lat) for lat,lon,_,_ in rows]
    names=[r[2] for r in rows]; grps=[r[3] for r in rows]
    return gpd.GeoDataFrame({"name":names,"group":grps}, geometry=geoms, crs="EPSG:4326")

def add_country_labels(ax, gdf_c, name_col="ADMIN", fontsize=5.5, color="#30343a",
                       overrides_ll=None,  # "Turkey": (lon, lat)
                       alpha=0.9):
    
    import geopandas as gpd
    from shapely.geometry import Point

    tr = {"Turkey":" ","Greece":"Yunanistan","Bulgaria":"Bulgaristan",
          "Georgia":"Gürcistan","Armenia":"Ermenistan","Azerbaijan":"Azerbaycan",
          "Syria":"Suriye","Iraq":"Irak","Iran":"İran"}
    overrides_ll = overrides_ll or {}

    for _, row in gdf_c.iterrows():
        if row.geometry.is_empty:
            continue
        name = row[name_col]
        label = tr.get(name, name)

        # If override exist use it
        if name in overrides_ll:
            lon, lat = overrides_ll[name]
            pt = gpd.GeoSeries([Point(lon, lat)], crs="EPSG:4326").to_crs(gdf_c.crs).iloc[0]
            px, py = pt.x, pt.y
        else:
        # If override not exist use representative_point()
            rp = row.geometry.representative_point()
            px, py = rp.x, rp.y

        ax.text(
            px, py, label,
            fontsize=fontsize, color=color, fontweight="bold",
            ha="center", va="center", alpha=alpha,
            path_effects=[pe.withStroke(linewidth=3, foreground="white", alpha=0.85)],
            clip_on=True, zorder=5
        )

# Parameters
LABEL_SIZE = 7.0
DX, DY = 14000, 9000    # start ofset
COLORS = {"a_class":"#1f77b4","b_class":"#cc0000","c_class":"#2ca02c","d_class":"#ac10e0"}
LEGEND_TXT = {"a_class":"a_class","b_class":"b_class","c_class":"c_class","d_class":"d_class"}
EDGE = "#0f0f10"

# Points
points = [
# a class
    (36.6538, 36.2391, "Reyhanlı (Cilvegözü) / Hatay", "a_class"),
    (42.5641, 37.1589, "Habur / Şırnak", "a_class"),
    (39.6540, 44.7996, "Dilucu / Iğdır", "a_class"),
    (41.9641, 26.6415, "Hamzabeyli / Edirne", "a_class"),
    (41.7181, 26.3489, "Kapıkule / Edirne", "a_class"),
    (41.9322, 27.3657, "Kırklareli", "a_class"),
    (40.9291, 26.3820, "İpsala / Edirne", "a_class"),
# c class
    (27.8408, 37.8489, "Aydın", "c_class"),
    (26.4088, 40.1350, "Çanakkale", "c_class"),
    (29.0860, 37.7761, "Denizli", "c_class"),
    (30.5203, 39.7703, "Eskişehir", "c_class"),
    (36.1592, 36.2031, "Hatay", "c_class"),
    (29.9448, 40.7660, "Kocaeli", "c_class"),
    (27.4298, 38.6142, "Manisa", "c_class"),
    (34.5980, 36.7852, "Mersin", "c_class"),
    (28.3583, 37.2076, "Muğla", "c_class"),
    (30.4031, 40.7773, "Sakarya", "c_class"),
    (27.5078, 40.9769, "Tekirdağ", "c_class"),
# b class
    (29.0324, 41.0943, "İstanbul", "b_class"),
    (27.1951, 38.4828, "İzmir", "b_class"),
    (32.8615, 39.9390, "Ankara", "b_class"),
    (30.8001, 36.9166, "Antalya", "b_class"),
# d_class
    (35.3933, 37.0032, "Adana", "d_class"),
    (35.5588, 38.7021, "Kayseri", "d_class"),
    (29.1018, 40.0785, "Bursa", "d_class"),
    (40.2597, 37.8737, "Diyarbakır", "d_class"),
    (39.7438, 40.9748, "Trabzon", "d_class"),
    (36.3435, 41.2593, "Samsun", "d_class"),
    (37.3571, 37.0038, "Gaziantep", "d_class"),
    (32.5407, 37.8888, "Konya", "d_class")
    ]
gdf_pts = make_gdf(points)

# Map data and cutting
world = load_world_10m()
name_col = "ADMIN" if "ADMIN" in world.columns else ("NAME_EN" if "NAME_EN" in world.columns else "name")
neighbors = ["Turkey","Greece","Bulgaria","Georgia","Armenia","Azerbaijan","Syria","Iraq","Iran"]
countries = world[world[name_col].isin(neighbors)].to_crs(3857)
tur = countries[countries[name_col]=="Turkey"]

pts3857 = gdf_pts.to_crs(3857)

# Turkey centered bbox
minx, miny, maxx, maxy = tur.to_crs(4326).total_bounds
padx, pady = 2.6, 2.0
bbox_4326 = gpd.GeoSeries([box(minx-padx, miny-pady, maxx+padx, maxy+pady)], crs=4326)
bbox_3857 = bbox_4326.to_crs(3857)
bxmin, bymin, bxmax, bymax = bbox_3857.total_bounds

# Country polygons cutting
countries_clip = gpd.clip(countries, bbox_3857.geometry.iloc[0])

# plotting
fig, ax = plt.subplots(figsize=(13, 9.2), dpi=320, facecolor="white")

# Country Polygons
countries_clip.plot(ax=ax, color="#f5faf5", edgecolor="#6b7078", linewidth=1.1, zorder=2)
countries_clip[countries_clip[name_col]=="Turkey"].plot(ax=ax, color="#f2f6fc", edgecolor="#3b4149", linewidth=1.4, zorder=3)

# Country Names
label_overrides = {
    "Turkey": (35.0, 39.0),
    "Greece": (24.5, 40.9),
    "Armenia": (44.7, 40.7)
    }  # Manuel Türkiye merkezi (lon, lat)

add_country_labels(
    ax,
    countries_clip,
    name_col=name_col,
    fontsize=12,
    color="#2b2f36",
    overrides_ll=label_overrides 
)

for grp, sub in pts3857.groupby("group"):
    sub.plot(ax=ax, markersize=30, color=COLORS.get(grp, "#1f77b4"),
             edgecolor=EDGE, linewidth=0.5, alpha=0.98, zorder=6)

# Labels
texts = []
for i, (x, y, name, grp) in enumerate(zip(
    pts3857.geometry.x,
    pts3857.geometry.y,
    pts3857["name"],
    pts3857["group"]
)):
    dx = DX if (i % 2 == 0) else -DX
    dy = DY if (i % 3 != 0) else -DY*0.6
    color = COLORS.get(grp, "#111")

    t = ax.text(
        x+dx, y+dy, name,
        fontsize=LABEL_SIZE,
        color=color,
        ha="left" if dx>0 else "right", va="bottom",
        path_effects=[pe.withStroke(linewidth=0.2, foreground="#1c0f0f", alpha=1)],
        clip_on=True,zorder=7
    )
    texts.append(t)

adjust_text(
    texts, ax=ax, only_move={'texts':'xy'},
    expand_points=(1.2, 1.4), expand_text=(1.1, 1.2),
    force_text=(0.9, 1.0),
    arrowprops=dict(arrowstyle="-", lw=1.4, color="#505760")
)

# Legend
handles = []
for key in ["a_class","b_class","c_class","d_class"]:
    handles.append(ax.scatter([], [], s=70, color=COLORS[key], edgecolor=EDGE, linewidth=0.5, label=LEGEND_TXT[key]))
leg = ax.legend(handles=handles, frameon=True, loc="lower left", fontsize=8.5)
leg.get_frame().set_alpha(0.92)

# Axisses and layouts
ax.set_xlim(bxmin, bxmax); ax.set_ylim(bymin, bymax)
ax.set_axis_off()
plt.tight_layout()

# Save
png_path = os.path.join(os.getcwd(), "turkey_points_final-v2.png")
svg_path = png_path.replace(".png", ".svg")
png_trans = png_path.replace(".png", "_transparent.png")

plt.savefig(png_path, dpi=320, bbox_inches="tight", pad_inches=0)              # opak
plt.savefig(png_trans, dpi=320, bbox_inches="tight", pad_inches=0, transparent=True)  # şeffaf
plt.savefig(svg_path, bbox_inches="tight", pad_inches=0)                       # vektör

plt.show()
print("Kaydedildi:\n- ", png_path, "\n- ", png_trans, "\n- ", svg_path)
